#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS = 3
   #
DB_NAME        = "my_db"
GRAPH_NAME     = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)


#  Betweeness algorithm

In [ ]:

my_graph.run(lambda g: g.nodes.remove_property("centrality"))

print("--")

#  AttributeError: 'katana_enterprise.distributed_native.Graph' object has no attribute 'nodes'


In [ ]:

from katana.remote import analytics

analytics.betweenness_centrality(
   my_graph,
      #
   result_property_name = "centrality",
   num_sources          = 100
   )

print("--")



In [ ]:

l_result = my_graph.query(
   f"""
    
   MATCH (n)
   RETURN n, n.centrality
   ORDER BY n.centrality DESC
   LIMIT 30
    
    """, contextualize=True,
   )

l_result.view()

